In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from shapely.geometry import Point, Polygon, MultiPoint
from shapely.wkt import loads
from sklearn.cluster import KMeans
import numpy as np

Import Data and Organize Columns, and remove bad data

In [ ]:
gdf = gpd.read_file("../data/SANGIS/BUSINESS_SITES/BUSINESS_SITES.shp")

In [ ]:
gdf = gdf[gdf['POINT_X']!=0]
gdf['x'] = gdf['geometry'].x
gdf['y'] = gdf['geometry'].y

Generate Hexbins
(note - gridsize was manually calculated based on width of SD County to generate roughly 1/4 mile radius hexbins:
SD County is roughly 86 miles east to west, and the grid size takes the quantity of hexbins by width
)

In [ ]:
hexes = matplotlib.pyplot.hexbin( x= gdf['x'], y=gdf['y'],mincnt=1,gridsize=86*2)

In [ ]:
hexbins = gpd.points_from_xy(x=[i[0] for i in hexes.get_offsets()],y=[i[1] for i in hexes.get_offsets()])[1:]

Merge the data with the newly generated hexbins

In [ ]:
left_merge = gpd.GeoDataFrame(hexbins, geometry=0)

In [ ]:
full_merge = gpd.sjoin_nearest(left_merge,gdf,how='right')

Count the # of datapoints in each hexbin

In [ ]:
index_and_counts = full_merge.groupby('index_left').count().sort_values(by='x').reset_index()[['index_left','APN',]]

In [ ]:
def get_x(index):
    return hexbins[index].x
def get_y(index):
    return hexbins[index].y

In [ ]:
index_and_counts['x'] = index_and_counts['index_left'].apply(get_x)
index_and_counts['y'] = index_and_counts['index_left'].apply(get_y)
index_and_counts['geometry'] = gpd.points_from_xy(index_and_counts['x'], index_and_counts['y'])

In [ ]:
df = gpd.GeoDataFrame(index_and_counts)

In [ ]:
df.plot(column='APN', markersize=2)

Clustering techniques - use maxima as cluster centers and run kmeans for distance

In [ ]:
# idea 1 - could we use 70 biggest maxima as centers?
df['is_center'] = df['APN']>=df['APN'].sort_values(ascending=False).reset_index(drop=True)[70]
df.plot(column='is_center')

In [ ]:
# clustering based off local maxima centers using kmeans
cluster_centers = df[df['is_center']==True][['x', 'y']].values
other_points = df[['x', 'y']].values
k = len(cluster_centers)
kmeans = KMeans(n_clusters=k, init=cluster_centers, n_init=1)
kmeans.fit(other_points)
df['cluster'] = kmeans.labels_

In [ ]:
# TODO: have at least 5 hexbins per cluster?/count of businesses per cluster?
df.plot(column='cluster', legend=True, markersize=2)

What is the difference from just clustering on distance

In [ ]:
# other idea - what does python have available? 
# basic dbscan clustering on distance
scaler = StandardScaler()
df['scaled_weight'] = scaler.fit_transform(df[['APN']])
features = df[['x', 'y','APN']]
dbscan = DBSCAN(eps=6000, min_samples=20)
df['cluster_label'] = dbscan.fit_predict(features)
df.plot(column='cluster_label', legend=True)

In [ ]:
# other idea - db scan but for just the counts to find local maxima?
scaler = StandardScaler()
df['scaled_weight'] = scaler.fit_transform(df[['APN']])
features = df[['APN']]
dbscan = DBSCAN(eps=1, min_samples=1)
df['cluster_label'] = dbscan.fit_predict(features)
df.plot(column='cluster_label', legend=True)

Generate Polygons from local maxima approach

In [ ]:
# TODO: potentially consolidate with block groups on intersects with hexbins
poly_df = full_merge.merge(df[['index_left','cluster']],how='left')

geometry = poly_df['geometry'].apply(Point)
gpdf = gpd.GeoDataFrame(poly_df, geometry=geometry)

grouped = gpdf.groupby('cluster')

polygons = []
for cluster, group in grouped:
    polygon = group['geometry'].unary_union.convex_hull
    polygons.append({'cluster': cluster, 'geometry': polygon})

polygons_gdf = gpd.GeoDataFrame(polygons)

polygons_gdf.plot(column='cluster')


In [ ]:
group['geometry'].unary_union

In [ ]:
blocks = gpd.read_file('../data/Census_Blocks_20231127.csv').drop(columns=['geometry'])
blocks['the_geom'] = blocks['the_geom'].apply(loads)
blocks = blocks.set_geometry('the_geom')

In [ ]:
blocks.explore()

In [ ]:
gdf[gdf['BUSTYPE'].str.contains('HOS')]

In [ ]:
gdf

In [ ]:
461/98162